#### Look at the "pre" Sales for a store

In [214]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from checkStationary import ADF
from checkSeasonality import powerDensity

PATH = '/home/tbrownex/data/Hackett/JLP/UseCase1/salesForecast/'
FILE = 'store_sales_by_OLG-pre.csv'

STORES = ["Twickenham", "Thame", "Wokingham", "Harrogate", "Chandlers Ford",\
          "Sidmouth", "Barry", "Banstead", "Westbury Park", "Monmouth"]

In [215]:
def printSeries(series, store):
    from matplotlib.ticker import MultipleLocator, FormatStrFormatter
    majorLocator = MultipleLocator(20)
    majorFormatter = FormatStrFormatter('%d')
    #minorLocator = MultipleLocator(1)
    
    fig, ax = plt.subplots()
    
    plt.plot(series)
    #plt.axvline(x=24,color='r', linewidth=.75)
    #plt.axvline(x=28,color='r', linewidth=.75)
    ax.set_ylim([0,series.max()])

    ax.xaxis.set_major_locator(majorLocator)
    ax.xaxis.set_major_formatter(majorFormatter)
# for the minor ticks, use no labels; default NullFormatter
    #ax.xaxis.set_minor_locator(minorLocator)
    plt.title(store)
    plt.show()

In [230]:
df = pd.read_csv(PATH+FILE)

##### Create Stationary and Seasonality table

In [9]:
print("{:<15}{:<13}{}".format("Store","Stationary","Seasonal"))
for store in STORES:
    tmp = df.loc[df["Business Unit Long Name"]==store]["Line Sales £"]
    tmp = tmp.reset_index(drop=True)
    stationary = ADF(tmp, "5%", False)
    
    freq = len(tmp)
    seasonal, maxFreq = powerDensity(series=tmp, freq=freq, store=None, plot=False)
    print("{:<18}{:<12}{}".format(store, str(stationary), seasonal))

Store          Stationary   Seasonal
Twickenham        True        False
Thame             True        False
Wokingham         True        False
Harrogate         True        False
Chandlers Ford    False       False
Sidmouth          True        False
Barry             True        False
Banstead          False       False
Westbury Park     True        False
Monmouth          True        False


##### Run g-h Filter against a store

In [74]:
def smooth(x,window_len=7,window='hanning'):
    s=np.r_[x[window_len-1:0:-1],x,x[-2:-window_len-1:-1]]
    if window == 'flat': #moving average
        w=np.ones(window_len,'d')
    else:
        w=eval('np.'+window+'(window_len)')

    y=np.convolve(w/w.sum(),s,mode='valid')
    return y

In [21]:
def ghFilter(series, init, rate, g, h, dt):
    estimates   = []
    predictions = []
    rates = []
    prevWeight = init
    for w in series:
        pred     = prevWeight + rate*dt
        residual = w - pred
        est      = pred + g*residual
        estimates.append(est)          # Save the estimate
        predictions.append(pred)       # Save the prediction
        rate = rate + h*(residual)/dt        # Update the model gain/day
        rates.append(rate)
        prevWeight = est
    return predictions, estimates, np.asarray(rates)

In [121]:
# Based on the last estimate and rate, generate forecast values
def getForecast(est, rate):
    forecast = []
    for x in range(1, 19):
        tmp = est + (x*rate)
        forecast.append(tmp)
    return forecast

In [130]:
def display(estimates, sales, store):
    plt.plot(estimates, color="r")
    plt.plot(sales, color="b")
    #plt.plot(predictions, color="y")
    plt.title(store)
    plt.savefig(store+".png", format="png")

In [217]:
# Set filter parameters
rate = 1.          # "dx"...the multiplier in the model
g    = .2
h    = .08
dt   = 1.            # time interval (1 day)

#store = input()
store = "Banstead"
sales    = df.loc[df["Business Unit Long Name"]==store]["Line Sales £"]
sales    = sales.reset_index(drop=True)

In [1]:
def generateEstimates(sales):
    init = sales[0]
    smoothed = smooth(sales,window_len=5)
    predictions, estimates, rates = ghFilter(smoothed, init, rate, g, h, dt)
    forecast = getForecast(estimates[-1], rates[-1])
    estimates = estimates+forecast
    estimates = pd.Series(x for x in estimates)
    return estimates

In [219]:
init = sales[0]
smoothed = smooth(sales,window_len=5)
predictions, estimates, rates = ghFilter(smoothed, init, rate, g, h, dt)
forecast = getForecast(estimates[-1], rates[-1])
estimates = estimates+forecast
estimates = pd.Series(x for x in estimates)

display(estimates, sales, store)
estimates.to_csv(PATH+store+".csv")